In [1]:
import json
import os
import pickle
import torch
import numpy as np

In [2]:
# Read JSON
def read_json(fname):
    with open(fname, "r") as handle:
        res = handle.read()
        obj = json.loads(res)
    return obj

In [3]:
# Read pickle
def read_pkl(fname):
    with open(fname, 'rb') as handle:
        obj_pkl = pickle.load(handle)
    return obj_pkl

In [4]:
# Read torch 
def read_torch(fname):
    obj = torch.load(fname)
    return obj

In [6]:
test_data = read_pkl('../data/zeshel/test_processed_data.pickle')

In [111]:
dictionary = read_pkl('data/zeshel/dictionary.pickle')

In [2]:
!ls data/zeshel/top64_cands

1nn.t7      1rand.t7    arbo.t7     in_batch.t7 knn.t7


In [3]:
!ls data/zeshel/results

bi_1nn.json         bi_in_batch.json    cross_1rand.json    cross_knn.json
bi_1rand.json       bi_knn.json         cross_arbo.json
bi_arbo.json        cross_1nn.json      cross_in_batch.json


In [15]:
existing_cands = read_torch('data/zeshel/top64_cands/in_batch.t7')
labels = np.array(existing_cands['labels'])
np.where(labels == -1)[0]

In [102]:
existing_cands = read_torch('data/zeshel/top64_cands/in_batch.t7')
exist_labels = np.array(existing_cands['labels'])
existing_idxs = np.where(exist_labels == -1)[0]
print(existing_idxs)

[   3    4    9 ... 9972 9987 9991]


In [103]:
our_cands = read_torch('data/zeshel/top64_cands/1rand.t7')
our_labels = np.array(our_cands['labels'])
our_idxs = np.where(our_labels != -1)[0]
our_idxs_bi_recallhits = np.where(our_labels == 0)[0]
our_idxs_bi_recallnonhits = np.where(our_labels > 0)[0]
print(len(our_idxs), " - ", our_idxs)
print(len(our_idxs_bi_recallhits), " - ", our_idxs_bi_recallhits)
print(len(our_idxs_bi_recallnonhits), " - ", our_idxs_bi_recallnonhits)

8570  -  [   0    1    2 ... 9997 9998 9999]
5086  -  [   0    5    6 ... 9997 9998 9999]
3484  -  [   1    2   10 ... 9991 9992 9993]


In [91]:
# existing model failed recall
# our biencoder linked correctly

print(len(np.intersect1d(existing_idxs, our_idxs_bi_hits)))
print(np.intersect1d(existing_idxs, our_idxs_bi_hits))

64
[ 220  249  526  669  694  810 1159 1389 1439 1591 1964 2342 2405 2640
 2675 2749 2789 2932 3100 3262 3336 3459 3830 3863 3900 3969 4259 4373
 4557 4785 4787 4846 5039 5745 5871 5888 6303 6347 6414 6448 6632 6731
 6896 6939 7001 7015 7312 7341 7437 7482 7525 8197 8262 8293 8331 8489
 9115 9368 9533 9648 9799 9867 9936 9972]


In [87]:
# existing model failed recall
# our biencoder recalled correctly
# our crossencoder linked correctly

exi_fail_ourbi_succ_idxs = np.intersect1d(existing_idxs, our_idxs_bi_recallnonhits)
print(exi_fail_ourbi_succ_idxs)

[  29   41   64   96  117  125  135  149  188  196  214  234  279  336
  350  385  387  394  408  427  456  464  489  524  533  546  547  553
  578  616  647  659  667  722  751  760  793  800  808  827  834  850
  852  883  897  906  917  924  940  966  968 1000 1003 1011 1015 1045
 1074 1088 1114 1135 1165 1169 1177 1205 1238 1245 1290 1299 1340 1352
 1394 1513 1563 1648 1667 1713 1715 1718 1742 1754 1816 1968 1978 1993
 1997 2000 2024 2049 2089 2115 2146 2168 2268 2294 2352 2363 2385 2430
 2445 2504 2512 2513 2535 2540 2548 2554 2576 2587 2645 2661 2665 2685
 2741 2751 2758 2776 2786 2787 2790 2852 2867 2869 2872 2881 2929 2937
 2948 2950 2957 3016 3026 3036 3065 3110 3126 3179 3215 3225 3250 3252
 3273 3345 3383 3411 3412 3436 3440 3473 3499 3506 3530 3540 3601 3617
 3623 3628 3637 3638 3659 3660 3668 3669 3672 3676 3684 3702 3756 3771
 3823 3844 3855 3858 3889 3896 3902 3926 3931 3955 3981 3999 4016 4032
 4034 4061 4064 4065 4083 4091 4106 4107 4108 4123 4148 4163 4164 4175
 4195 

In [76]:
cross_res = read_json('data/zeshel/results/cross_1rand.json')
success_men_ids = set([s['mention_id'] for s in cross_res['success']])

In [79]:
cross_succ_idxs = [i for i in range(len(test_data)) if test_data[i]['mention_id'] in success_men_ids]

In [100]:
final_idxs = np.intersect1d(exi_fail_ourbi_succ_idxs, cross_succ_idxs)
print(len(final_idxs), " - ", final_idxs)

133  -  [  64  135  385  387  394  408  524  533  578  659  793  808  966  968
 1045 1074 1088 1238 1290 1352 1715 1718 1754 2024 2049 2168 2268 2445
 2504 2513 2645 2661 2741 2751 2852 2869 2937 3026 3215 3250 3252 3383
 3436 3473 3506 3637 3638 3659 3668 3771 3855 3858 3889 3902 3931 4034
 4065 4083 4148 4209 4245 4296 4430 4441 4496 4772 4922 5047 5181 5207
 5212 5388 5481 5548 5754 5767 5868 5901 6038 6046 6327 6385 6435 6592
 6606 6619 6804 6889 6891 6901 7254 7288 7305 7309 7421 7436 7621 7674
 7737 7750 7842 7927 7986 8077 8118 8155 8227 8340 8348 8418 8426 8493
 8546 8625 8677 8868 8876 9030 9073 9148 9156 9167 9493 9517 9575 9596
 9608 9679 9697 9906 9930 9940 9957]


In [115]:
res = []

menid_to_idx = {c['mention_id']: i for i,c in enumerate(cross_res['success'])}
for i in final_idxs:
    d = test_data[i]
    men_id = d['mention_id']
    ctxt = ' '.join(d['context']['tokens'])
    etype = d['type']
    name = d['mention_name']
    correct_label_idx = d['label_idxs'][0]
    existing_label_idx = existing_cands['candidates'][i][0]  # Existing model's best prediction
    our_label_idx = our_cands['candidates'][i][0]  # Our biencoder model's best prediction
    res.append({
        'mention_id': men_id,
        'test_set_idx': i,
        'type': etype,
        'name': name,
        'context': ctxt,
        'existing_cui': dictionary[existing_label_idx]['cui'],
        'existing_cui_name': dictionary[existing_label_idx]['title'],
        'existing_cui_desc': ' '.join(dictionary[existing_label_idx]['tokens']),
        'our_bi_cui': dictionary[our_label_idx]['cui'],
        'our_bi_cui_name': dictionary[our_label_idx]['title'],
        'our_bi_cui_desc': ' '.join(dictionary[our_label_idx]['tokens']),
        'our_cross_cui': cross_res['success'][menid_to_idx[men_id]]['mention_gold_cui'],
        'our_cross_cui_name': cross_res['success'][menid_to_idx[men_id]]['mention_gold_cui_name'],
        'our_cross_cui_desc': ' '.join(dictionary[correct_label_idx]['tokens']),
    })

In [ ]:
np.random.shuffle(res)

In [110]:
list(np.array(test_data)[final_idxs])

[{'mention_id': '0195C8FCA206B09C',
  'mention_name': 'his Deck',
  'context': {'tokens': ['[CLS]',
    'for',
    'three',
    'minutes',
    'during',
    'which',
    'maintenance',
    'was',
    'carried',
    'out',
    '.',
    'in',
    'the',
    'days',
    'coming',
    'up',
    'to',
    'the',
    'scheduled',
    'maintenance',
    ',',
    'yu',
    '##sei',
    'worked',
    'on',
    'his',
    'duel',
    'runner',
    'and',
    'made',
    'sure',
    'it',
    'could',
    'cover',
    'the',
    'necessary',
    'distance',
    'in',
    'three',
    'minutes',
    '.',
    'before',
    'yu',
    '##sei',
    'left',
    ',',
    'rally',
    'gave',
    'him',
    'the',
    'card',
    '"',
    'turbo',
    'booster',
    '"',
    ',',
    'which',
    'yu',
    '##sei',
    'temporarily',
    'added',
    'to',
    '[unused1]',
    'his',
    'deck',
    '[unused2]',
    '.',
    'on',
    'the',
    'day',
    ',',
    'sector',
    'security',
    'detected

In [75]:
cross_res['success']

[{'mention_id': '0004DD84239096E0',
  'mention_name': 'Vault of Gnashing Teeth',
  'mention_gold_cui': 'A0604B5BC7C4CAAF',
  'mention_gold_cui_name': 'Moondeep Sea',
  'predicted_name': 'Moondeep Sea',
  'predicted_cui': 'A0604B5BC7C4CAAF'},
 {'mention_id': '0011676B60CA3A4C',
  'mention_name': 'native to the planet',
  'mention_gold_cui': 'F011C6DD6179D8C7',
  'mention_gold_cui_name': 'Sarpeidon native',
  'predicted_name': 'Sarpeidon native',
  'predicted_cui': 'F011C6DD6179D8C7'},
 {'mention_id': '0040F6C2D35D6A00',
  'mention_name': 'Vengeance',
  'mention_gold_cui': '3187B593524A6D86',
  'mention_gold_cui_name': 'USS Vengeance',
  'predicted_name': 'USS Vengeance',
  'predicted_cui': '3187B593524A6D86'},
 {'mention_id': '0043D48466026BB0',
  'mention_name': 'Royale Hotel',
  'mention_gold_cui': '4D99889452DB6CAD',
  'mention_gold_cui_name': 'Royale',
  'predicted_name': 'Royale',
  'predicted_cui': '4D99889452DB6CAD'},
 {'mention_id': '0046D0B545F1A899',
  'mention_name': 'the pre

In [112]:
dictionary[0]

{'cui': '000523A4D586C293',
 'title': 'Warner',
 'description': 'Warner Warner was a communications technician aboard Nerva Beacon . He was responsible for redirecting Pluto - Earth Flight One - Five from doing a dropover on the Nerva , which was under quarantine because of an apparent plague infection killing off the crew . Not long after this , he was bitten by a Cybermat and infused with its venom , the real culprit behind the crew \' s deaths . The Fourth Doctor , Sarah Jane Smith , Harry Sullivan and the surviving Nerva crew failed to save him . Despite his strong constitution , Warner died from the effects of the venom after his pulse and temperature shot up beyond what could support human life , anguishing on longer than most of the other infected victims . ( TV : " Revenge of the Cybermen " )',
 'type': 'doctor_who',
 'tokens': ['[CLS]',
  'warner',
  '[unused3]',
  'warner',
  'was',
  'a',
  'communications',
  'technician',
  'aboard',
  'ne',
  '##rva',
  'beacon',
  '.',
 

In [129]:
types = {'forgotten_realms':[], 'lego':[], 'star_trek':[], 'yugioh':[]}
for i,r in enumerate(res):
    types[r["type"]].append(i)

In [137]:
res[types['forgotten_realms'][0]]

{'mention_id': 'F5031411EF351EA5',
 'test_set_idx': 9575,
 'type': 'forgotten_realms',
 'name': 'Saukuruk',
 'context': '[CLS] they did not believe in life after death , and they believed that the world was not created — it simply always was . a very small number of i ##ulu ##ti ##un ##s were exceptions to this rule . among this tiny population of adherents were followers of pa ##hl ##uru ##k , a ##yu ##ru ##k , and [unused1] sa ##uk ##uru ##k [unused2] . such i ##ulu ##ti ##un ##s were considered strange by the rest of their society . however , i ##ulu ##ti ##un ##s " did " follow a sort of world view or philosophy called " qu ##ko ##ko " , which held that all living creatures shared the same essence of life called " ea ##as " . [SEP]',
 'existing_cui': 'C87DB9E9D6BA5F31',
 'existing_cui_name': 'Iulutiun',
 'existing_cui_desc': '[CLS] i ##ulu ##ti ##un [unused3] the i ##ulu ##ti ##un ##s ( ee - o ##o - lo ##o - she ##e - un ##s ) were a human people living in the al ##pu ##k region of

In [140]:
# Case handled: "specific type of the more general surface form concept"
res[types['forgotten_realms'][3]]

{'mention_id': 'BA7B7AD7AC9791CC',
 'test_set_idx': 7288,
 'type': 'forgotten_realms',
 'name': 'sword',
 'context': '[CLS] give him a day off or have him retire immediately , letting him be with his family but losing his services as a party member . ke ##ld ##orn is a fair fighter , and as an experienced and esteem ##ed pal ##adi ##n comes equipped with perhaps the best items of any np ##c ; he has both an enchanted [unused1] sword [unused2] and plate mail that only he can use . as an in ##qui ##sitor pal ##adi ##n , he also has the powerful ability to cast " di ##sp ##el magic " at twice his own level . he \' s one of the older np ##cs in the game , often appearing as a wise and experienced father figure of sorts [SEP]',
 'existing_cui': '337BB2AE0D26B7EB',
 'existing_cui_name': 'Sword',
 'existing_cui_desc': '[CLS] sword [unused3] a sword is a melee weapon that in its simplest form consists of a blade and a hilt . this definition includes those weapons that fit the arch ##ety ##pe o

In [151]:
# Case handled: Typo in mention surface form
res[types['forgotten_realms'][14]]

{'mention_id': '2BB5369961539D7E',
 'test_set_idx': 1754,
 'type': 'forgotten_realms',
 'name': '" Chacker " and " Whacker "',
 'context': '[CLS] the hall • gut ##bus ##ter brigade • harp ##ell • house bae ##nr ##e • house barr ##ison del \' arm ##go • house do \' ur ##den • house mi ##zz ##ry ##m • house x ##or ##lar ##rin • knights in silver • wild ##d ##war ##f brigade artifacts . bag of holding • [unused1] " cha ##cker " and " w ##ha ##cker " [unused2] • fae ##rz ##ress • " horn of val ##hall ##a " • " ic ##ing ##de ##ath " • " k ##ha ##zi ##d \' he ##a " • " ko ##za ##h \' s needle " • " orb ##bc ##ress " • " ring of elemental power " • " robe of the arch [SEP]',
 'existing_cui': 'A11E7DA71FDAD157',
 'existing_cui_name': 'Wyverntongue Chalice',
 'existing_cui_desc': "[CLS] w ##y ##vern ##ton ##gue cha ##lice [unused3] the w ##y ##vern ##ton ##gue cha ##lice was one of the relics of the nine . it contained the spirit of tre ##th halo ##nte ##r who could be called forth by the cha #

In [169]:
res[74]

{'mention_id': '5C34478AE15A0FB1',
 'test_set_idx': 3638,
 'type': 'star_trek',
 'name': 'their ship',
 'context': '[CLS] according to the script of , there were two and ##oria ##n moons . however , this was later changed to two ten ##ee ##bian moons for the filming of the episode . in the final draft script of ( dated ) , and ##oria was referred to as having once been visited by travis may ##weather and his family aboard [unused1] their ship [unused2] . however , this reference was later changed to la ##vin ##ius ( by or on , when a revised final draft of the script was issued ) , and the scene was ultimately deleted . in contrast with the script of " broken bow " referring to and ##oria as having a pair of moons , attempted to establish the [SEP]',
 'existing_cui': 'EE4A583FAE1437B8',
 'existing_cui_name': 'Andorian renegade ship',
 'existing_cui_desc': "[CLS] and ##oria ##n renegade ship [unused3] the and ##oria ##n renegade ship was operated by renegade and ##oria ##n s in the tri 

In [170]:
res[94]

{'mention_id': 'CEDB3DABB29B4B78',
 'test_set_idx': 8077,
 'type': 'star_trek',
 'name': 'something green',
 'context': '[CLS] . both had just returned from 1893 \' s san francisco and clemens was just telling the counselor about his disappointment as there were no cigar s stocked on the " enterprise " - d . ( ) he was having a drink and chatting with a bartender in ten forward when montgomery scott entered the bar and later drank " [unused1] something green [unused2] " . ( ) wallace worked in a transport ##er room when the " enterprise " - d was in orbit of be ##rsa ##llis iii . he beamed aboard doctor crush ##er and the infirmary patients of the be ##rsa ##llis colony which had to be evacuated due to massive fires ##torm s . ( ) later that year , [SEP]',
 'existing_cui': '5CCB83C71D089E29',
 'existing_cui_name': 'Green tea',
 'existing_cui_desc': "[CLS] green tea [unused3] green tea was a type of tea made with tea leaves that had undergone little oxidation . it was an exceptional sou

In [171]:
res[107]

{'mention_id': '8F0A816B65D0BB77',
 'test_set_idx': 5548,
 'type': 'star_trek',
 'name': 'father',
 'context': '[CLS] , several inmates wearing ankle monitor s are performing labor . one man \' s work is interrupted when an authoritative - looking woman in a star ##fleet uniform approaches him and asks , " tom paris ? " the man glances up at her and she introduces herself as kathryn jane ##way . she states that she served with his [unused1] father [unused2] on the and that she would like to speak with him about a job she wants him to do . he sarcastically replies that he is already doing a job for the federation , but jane ##way says that the rehab commission is very pleased with his work and has given her permission to discuss the matter with him . he replies [SEP]',
 'existing_cui': 'A412005AAAEAC0A3',
 'existing_cui_name': 'Janeway ( Male Admiral )',
 'existing_cui_desc': '[CLS] jane ##way ( male admiral ) [unused3] vice admiral jane ##way was a star ##fleet flag officer and father 

In [172]:
res[119]

{'mention_id': 'D740D83B7F68D61F',
 'test_set_idx': 8418,
 'type': 'star_trek',
 'name': 'his shuttle',
 'context': '[CLS] of mor ##dan iv displayed several weapons on a wall behind his desk in 236 ##4 . among them were two phase ##rs , one a star ##fleet type 2 phase ##r . ( ) phase ##rs of this type remained in sporadic use into the 24th century ; dir ##go carried several 23rd - century type 2 phase ##rs aboard [unused1] his shuttle [unused2] in 236 ##7 . ( ) 24th century and beyond . by the 236 ##0s , the type 2 phase ##r had evolved past the pistol - like configuration , becoming a sleek , silver - colored device with a handle flowing into the form of a muzzle cow ##ling and emi ##tter . with two small buttons for beam width [SEP]',
 'existing_cui': '01C23B9DFEB9BEEC',
 'existing_cui_name': 'Space shuttle',
 'existing_cui_desc': '[CLS] space shuttle [unused3] a space shuttle was a form of low - orbit spacecraft commonly launched from earth during the 20th and 21st centuries . the t

In [173]:
res[126]

{'mention_id': '2AA8A24C22D9696B',
 'test_set_idx': 1715,
 'type': 'star_trek',
 'name': 'Klingon ship',
 'context': "[CLS] , but allowed them to get out and conduct repairs to the outer hull of ska ##la ##ar ' s shuttle without environmental suit s . ( ) ind ##ri viii was a class l planet that lacked animal life but used to be covered with deciduous plants until 236 ##9 . that year , the had to witness a [unused1] k ##ling ##on ship [unused2] destroying the planet ' s lower atmosphere including all life with a plasma reaction . ( ) the crashed on a class l planet in the ruth ##arian sector in 237 ##1 . the sole survivor was able to survive in a cave on the planet ' s surface for a while , but died when she ran out of [SEP]",
 'existing_cui': '123E018548D4EEBC',
 'existing_cui_name': 'Klingon shuttlecraft',
 'existing_cui_desc': '[CLS] k ##ling ##on shuttle ##craft [unused3] the k ##ling ##on shuttle ##craft was a type of small craft in use by the k ##ling ##on s during the mid - 22nd 

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

for i in types['star_trek']:
    print(f"Index: {i}\n")
    pp.pprint(res[i])
    print("\n")

In [8]:
selected = [
{'mention_id': 'F5031411EF351EA5',
 'test_set_idx': 9575,
 'type': 'forgotten_realms',
 'name': 'Saukuruk',
 'context': '[CLS] they did not believe in life after death , and they believed that the world was not created — it simply always was . a very small number of i ##ulu ##ti ##un ##s were exceptions to this rule . among this tiny population of adherents were followers of pa ##hl ##uru ##k , a ##yu ##ru ##k , and [unused1] sa ##uk ##uru ##k [unused2] . such i ##ulu ##ti ##un ##s were considered strange by the rest of their society . however , i ##ulu ##ti ##un ##s " did " follow a sort of world view or philosophy called " qu ##ko ##ko " , which held that all living creatures shared the same essence of life called " ea ##as " . [SEP]',
 'existing_cui': 'C87DB9E9D6BA5F31',
 'existing_cui_name': 'Iulutiun',
 'existing_cui_desc': '[CLS] i ##ulu ##ti ##un [unused3] the i ##ulu ##ti ##un ##s ( ee - o ##o - lo ##o - she ##e - un ##s ) were a human people living in the al ##pu ##k region of the great glacier . they were one of the ethnic ##ities collectively known as the ul ##uti ##un s and the most numerous of all human tribes living on the glacier . description . i ##ulu ##ti ##un ##s were short and stock ##y and had round and flat facial features . they rarely were taller than five feet ( 150 cm ) . males usually weighed between 130 and 260 pounds ( 60 – 120 kg ) , and females were about 35 pounds ( 16 kg [SEP]',
 'our_bi_cui': '407FFA8C10F603FC',
 'our_bi_cui_name': 'Nakulutiun',
 'our_bi_cui_desc': '[CLS] nak ##ulu ##ti ##un [unused3] the nak ##ulu ##ti ##un ##s were a rec ##lus ##ive people residing in the nak ##val ##iga ##ch region of the great glacier , the most in ##hos ##pit ##able area of that frozen wilderness . they were the smallest ( only 5 % ) of the tribes collectively called the ul ##uti ##un s . description . nak ##ulu ##ti ##un ##s were short like all ul ##uti ##un ##s , having round and flat facial features , but they were thinner . they rarely were taller than five feet ( 150 cm ) . males usually weighed between 130 and 260 pounds ( 60 – 120 kg ) , and females were about 35 pounds ( 16 kg [SEP]',
 'our_cross_cui': 'AD3EF84B3795FDDA',
 'our_cross_cui_name': 'Auril',
 'our_cross_cui_desc': '[CLS] au ##ril [unused3] au ##ril ( pronounced " aw - ri ##ll " ) , also known as the cold goddess , the frost ##maid ##en , iced ##aw ##n , and the goddess of winter , was a fi ##ckle , vain , and evil deity , who was primarily ve ##ner ##ated out of fear . the few among the i ##ulu ##ti ##un s of the great glacier who worship ##ed her called her sa ##uk ##uru ##k . relationships . au ##ril , along with mala ##r , um ##ber ##lee , and their superior tal ##os , was a member of the deities of fury . um ##ber ##lee and au ##ril cooperate ##d with some degree of confidence , while [SEP]'},
 {'mention_id': 'BA7B7AD7AC9791CC',
 'test_set_idx': 7288,
 'type': 'forgotten_realms',
 'name': 'sword',
 'context': '[CLS] give him a day off or have him retire immediately , letting him be with his family but losing his services as a party member . ke ##ld ##orn is a fair fighter , and as an experienced and esteem ##ed pal ##adi ##n comes equipped with perhaps the best items of any np ##c ; he has both an enchanted [unused1] sword [unused2] and plate mail that only he can use . as an in ##qui ##sitor pal ##adi ##n , he also has the powerful ability to cast " di ##sp ##el magic " at twice his own level . he \' s one of the older np ##cs in the game , often appearing as a wise and experienced father figure of sorts [SEP]',
 'existing_cui': '337BB2AE0D26B7EB',
 'existing_cui_name': 'Sword',
 'existing_cui_desc': '[CLS] sword [unused3] a sword is a melee weapon that in its simplest form consists of a blade and a hilt . this definition includes those weapons that fit the arch ##ety ##pe of blade with the additional restriction of a minimum size . for game purposes , weapons are divided into three size / weight classes : light , one - handed , and two - handed . all swords are in either the one - handed or the two - handed class ( or both ) with the exception of the shorts ##word which is classified as a light weapon . it is included here mainly because of its widespread use and because it is used by smaller races in the same fashion as the [SEP]',
 'our_bi_cui': '4390344C57F338AD',
 'our_bi_cui_name': 'Warblade ( weapon )',
 'our_bi_cui_desc': "[CLS] war ##bla ##de ( weapon ) [unused3] the war ##bla ##de ( also known as the noble warrior ' s blade , or ar ##y ' ve ##lah ##r ' ke ##ry ##m in el ##ven ) , was one of three elf ##bla ##de s created in ancient co ##rman ##thy ##r to help the elves establish a ruling family . the war ##bla ##de was used to name the arms - major , the defender by arm , as the wi ##eld ##er is called . the bare ##r of the war ##bla ##de was responsible for the primary armed forces of the realm at large , known as the ak ##h ' ve ##lah ##r . history . the war ##bla ##de was [SEP]",
 'our_cross_cui': '32C8D16B4D6CF86C',
 'our_cross_cui_name': 'Hallowed Redeemer',
 'our_cross_cui_desc': '[CLS] hall ##owed red ##eem ##er [unused3] the hall ##owed red ##eem ##er was a special two - handed sword owned by the pal ##adi ##n ke ##ld ##orn fire ##cam . history . he received it as a gift from tor ##m himself early on in his career , and no - one else could use it . he was wielding it in 136 ##9 dr . a bastard sword called hall ##owed red ##eem ##er surfaced in the 137 ##0s dr with a different appearance and a very different en ##chan ##tment and usable by others . powers . in addition to being a very accurate " + 2 sword " , the hall ##owed red ##eem ##er protected ke ##ld ##orn with a special [SEP]'},
 {'mention_id': '2BB5369961539D7E',
 'test_set_idx': 1754,
 'type': 'forgotten_realms',
 'name': '" Chacker " and " Whacker "',
 'context': '[CLS] the hall • gut ##bus ##ter brigade • harp ##ell • house bae ##nr ##e • house barr ##ison del \' arm ##go • house do \' ur ##den • house mi ##zz ##ry ##m • house x ##or ##lar ##rin • knights in silver • wild ##d ##war ##f brigade artifacts . bag of holding • [unused1] " cha ##cker " and " w ##ha ##cker " [unused2] • fae ##rz ##ress • " horn of val ##hall ##a " • " ic ##ing ##de ##ath " • " k ##ha ##zi ##d \' he ##a " • " ko ##za ##h \' s needle " • " orb ##bc ##ress " • " ring of elemental power " • " robe of the arch [SEP]',
 'existing_cui': 'A11E7DA71FDAD157',
 'existing_cui_name': 'Wyverntongue Chalice',
 'existing_cui_desc': "[CLS] w ##y ##vern ##ton ##gue cha ##lice [unused3] the w ##y ##vern ##ton ##gue cha ##lice was one of the relics of the nine . it contained the spirit of tre ##th halo ##nte ##r who could be called forth by the cha ##lice ' s owner . tre ##th ' s spirit was forced to obey the commands of the owner . the cha ##lice was displayed in the royal palace in su ##zai ##l for many years but was stolen as a drunken prank by net ##hg ##las storms ##er ##pen ##t sometime in the 147 ##0s dr . when the cha ##lice ' s theft stirred the war wizard s into a frenzy , net ##hg ##las became worried and st ##ash ##ed [SEP]",
 'our_bi_cui': '5FED53882EFD0FAD',
 'our_bi_cui_name': 'Standing deer',
 'our_bi_cui_desc': '[CLS] standing deer [unused3] the standing deer was a magical cha ##lice . it was one of the wo - ha ui - jung , a collection of magical vessels created by the han people of ko ##ry ##o in ancient times . description . it was carved in the shape of a deer and had a deep cup . powers . the magic of the " standing deer " was invoked by filling it with rose petals . it could be used to cure blindness and to cast the shu ##ken ##ja spell " commune with lesser spirit " . handling the cha ##lice caused its g ##lam ##or to rub off on the holder , improving their social status as with the spell " face [SEP]',
 'our_cross_cui': '4D6355EC1018E2A8',
 'our_cross_cui_name': 'Cracker and Whacker',
 'our_cross_cui_desc': '[CLS] crack ##er and w ##ha ##cker [unused3] crack ##er and w ##ha ##cker were twin morning star s that were the chief weapons of the dwarf at ##hr ##oga ##te . description . " crack ##er " and " w ##ha ##cker " were both gray glass ##tee ##l morning star s , with spiked metal heads attached to the ends of adamant ##ine chains . at ##hr ##oga ##te generally wi ##eld ##ed " crack ##er " in the left hand and " w ##ha ##cker " in the right hand . powers . on command , " crack ##er " emitted from the small nu ##bs on the striking ball a dark reddish and chalk ##y fluid , the ich ##or of a rust [SEP]'},
 {'mention_id': '5C34478AE15A0FB1',
 'test_set_idx': 3638,
 'type': 'star_trek',
 'name': 'their ship',
 'context': '[CLS] according to the script of , there were two and ##oria ##n moons . however , this was later changed to two ten ##ee ##bian moons for the filming of the episode . in the final draft script of ( dated ) , and ##oria was referred to as having once been visited by travis may ##weather and his family aboard [unused1] their ship [unused2] . however , this reference was later changed to la ##vin ##ius ( by or on , when a revised final draft of the script was issued ) , and the scene was ultimately deleted . in contrast with the script of " broken bow " referring to and ##oria as having a pair of moons , attempted to establish the [SEP]',
 'existing_cui': 'EE4A583FAE1437B8',
 'existing_cui_name': 'Andorian renegade ship',
 'existing_cui_desc': "[CLS] and ##oria ##n renegade ship [unused3] the and ##oria ##n renegade ship was operated by renegade and ##oria ##n s in the tri ##ang ##ulum system . this ship was dismantled by its occupants and hidden , leading a star ##fleet admiral to believe they had vacated the star system . jean - luc pic ##ard recalled this incident in 236 ##6 , comparing it to wo ##rf ' s inability to find any trace of those who had attacked the delta rana iv colony . ( ) [SEP]",
 'our_bi_cui': 'EE4A583FAE1437B8',
 'our_bi_cui_name': 'Andorian renegade ship',
 'our_bi_cui_desc': "[CLS] and ##oria ##n renegade ship [unused3] the and ##oria ##n renegade ship was operated by renegade and ##oria ##n s in the tri ##ang ##ulum system . this ship was dismantled by its occupants and hidden , leading a star ##fleet admiral to believe they had vacated the star system . jean - luc pic ##ard recalled this incident in 236 ##6 , comparing it to wo ##rf ' s inability to find any trace of those who had attacked the delta rana iv colony . ( ) [SEP]",
 'our_cross_cui': '6DE52AD14E56461C',
 'our_cross_cui_name': 'ECS Horizon',
 'our_cross_cui_desc': '[CLS] ec ##s horizon [unused3] the ec ##s " horizon " was a freighter commissioned in 210 ##2 , of the earth cargo service under the command of paul may ##weather . history . in 212 ##6 , travis may ##weather was born aboard the " horizon " halfway between dr ##ay ##la ##x and vega colony . ( ) in 214 ##1 , the " horizon " encountered a delta ##n ship . travis may ##weather , who was fifteen at the time , described the delta ##n s as very attractive and very open . ( ) the " horizon " once came under attack by na ##us ##ica ##an pirate s ; it barely escaped in one piece . ( ) in january 215 [SEP]'},
 {'mention_id': 'CEDB3DABB29B4B78',
 'test_set_idx': 8077,
 'type': 'star_trek',
 'name': 'something green',
 'context': '[CLS] . both had just returned from 1893 \' s san francisco and clemens was just telling the counselor about his disappointment as there were no cigar s stocked on the " enterprise " - d . ( ) he was having a drink and chatting with a bartender in ten forward when montgomery scott entered the bar and later drank " [unused1] something green [unused2] " . ( ) wallace worked in a transport ##er room when the " enterprise " - d was in orbit of be ##rsa ##llis iii . he beamed aboard doctor crush ##er and the infirmary patients of the be ##rsa ##llis colony which had to be evacuated due to massive fires ##torm s . ( ) later that year , [SEP]',
 'existing_cui': '5CCB83C71D089E29',
 'existing_cui_name': 'Green tea',
 'existing_cui_desc': "[CLS] green tea [unused3] green tea was a type of tea made with tea leaves that had undergone little oxidation . it was an exceptional source of anti ##ox ##ida ##nt s , al ##kal ##oid s , and amino acid s . ( ) in 215 ##1 , sub - commander t ' pol came to the mess hall late at night for a cup of hot green tea . ( ) in 225 ##6 , michael burnham had a cup of green tea while con ##vers ##ing with about sar ##ek . ( ) [SEP]",
 'our_bi_cui': '2A887C680DA92880',
 'our_bi_cui_name': 'Unnamed beverages ( 23rd century )',
 'our_bi_cui_desc': '[CLS] unnamed beverages ( 23rd century ) [unused3] the following is a list of unnamed beverage s consumed during the 23rd century . green drink . in 226 ##8 , montgomery scott offered this green drink to the ke ##lva ##n tom ##ar , which he claimed to have found on gan ##yme ##de . unable to identify it by name after examining it when tom ##ar asked , scott simply replied , " " it \' s . " " ( ) york ##town drinks . the following drinks were consumed at york ##town in 226 ##3 of the alternate reality . jay ##lah \' s drinks . " jay ##lah was told by somebody that various drinks would take the edge off , so she [SEP]',
 'our_cross_cui': 'B70236171A1DE4E8',
 'our_cross_cui_name': 'Aldebaran whiskey',
 'our_cross_cui_desc': '[CLS] al ##de ##bara ##n whiskey [unused3] al ##de ##bara ##n whiskey was a strong , green alcohol ic beverage , favored by gui ##nan , who kept a bottle behind the bar in ten forward . it was a gift from her friend , jean - luc pic ##ard . in 236 ##9 , data served montgomery scott the drink of al ##de ##bara ##n whiskey , although he was unsure of its characteristics beyond its color . scott ##y found this beverage quite satisfying and drank most of the bottle himself , although he shared a couple of glasses with pic ##ard . ( ) in 237 ##1 , qu ##ark discovered an empty bottle of al ##de ##bara ##n whiskey in his brother rom [SEP]'},
 {'mention_id': '8F0A816B65D0BB77',
 'test_set_idx': 5548,
 'type': 'star_trek',
 'name': 'father',
 'context': '[CLS] , several inmates wearing ankle monitor s are performing labor . one man \' s work is interrupted when an authoritative - looking woman in a star ##fleet uniform approaches him and asks , " tom paris ? " the man glances up at her and she introduces herself as kathryn jane ##way . she states that she served with his [unused1] father [unused2] on the and that she would like to speak with him about a job she wants him to do . he sarcastically replies that he is already doing a job for the federation , but jane ##way says that the rehab commission is very pleased with his work and has given her permission to discuss the matter with him . he replies [SEP]',
 'existing_cui': 'A412005AAAEAC0A3',
 'existing_cui_name': 'Janeway ( Male Admiral )',
 'existing_cui_desc': '[CLS] jane ##way ( male admiral ) [unused3] vice admiral jane ##way was a star ##fleet flag officer and father of two daughters . one of his daughters , kathryn , was the former captain of the . the admiral died under the ice cap of tau ce ##ti prime in 235 ##8 . ( ) his widow was still alive as of 237 ##8 . ( ) when kathryn was a child , jane ##way and his wife took kathryn and her sister on backpack ##ing trips . they considered it was important that they should all keep a connection to their pioneer roots . similarly , jane ##way and kathryn hike ##d the north rim of the grand canyon when the latter was nine years [SEP]',
 'our_bi_cui': 'BC22C5D0072411A8',
 'our_bi_cui_name': "Michael O ' Brien",
 'our_bi_cui_desc': "[CLS] michael o ' brien [unused3] michael o ' brien was the father of miles o ' brien and at least two other sons . ( ; ) michael o ' brien was once invited on board the sometime prior to 236 ##7 , a stars ##hip on which his son miles was serving on . he later embarrassed miles by chasing nurse stanton around a bio ##bed in sick ##bay . ( ) michael occasionally span ##ked miles when he mis ##be ##ha ##ved . when his wife was pregnant , michael would give her massage s to light ##en her mood . ( ) michael hoped that his son would become a cello player . michael sent a recorded audition to the al ##de ##bara [SEP]",
 'our_cross_cui': 'E165AD0661BADCFC',
 'our_cross_cui_name': 'Owen Paris',
 'our_cross_cui_desc': "[CLS] owen paris [unused3] admiral owen paris was a senior star ##fleet official , the former captain of the stars ##hip , and the father of tom paris . ( ) his most significant contribution was in command of the path ##finder project and its follow - up , operation watson , which developed major advances in long - range communication to successfully rees ##ta ##bl ##ish contact with the lost star ##fleet vessel . history . owen paris held several high ranking posts in star ##fleet from captain to admiral , but also had a family : he was married and had children . ( ) owen was an instructor at star ##fleet academy , having taught his son ' s course on survival strategies in [SEP]"},
 {'mention_id': 'D740D83B7F68D61F',
 'test_set_idx': 8418,
 'type': 'star_trek',
 'name': 'his shuttle',
 'context': '[CLS] of mor ##dan iv displayed several weapons on a wall behind his desk in 236 ##4 . among them were two phase ##rs , one a star ##fleet type 2 phase ##r . ( ) phase ##rs of this type remained in sporadic use into the 24th century ; dir ##go carried several 23rd - century type 2 phase ##rs aboard [unused1] his shuttle [unused2] in 236 ##7 . ( ) 24th century and beyond . by the 236 ##0s , the type 2 phase ##r had evolved past the pistol - like configuration , becoming a sleek , silver - colored device with a handle flowing into the form of a muzzle cow ##ling and emi ##tter . with two small buttons for beam width [SEP]',
 'existing_cui': '01C23B9DFEB9BEEC',
 'existing_cui_name': 'Space shuttle',
 'existing_cui_desc': '[CLS] space shuttle [unused3] a space shuttle was a form of low - orbit spacecraft commonly launched from earth during the 20th and 21st centuries . the term " space shuttle " remained in use well into the 23rd century , when in 228 ##5 , sp ##ock was alerted by a public announcement that his space shuttle would be leaving in fifteen minute s . his destination was the . ( ) [SEP]',
 'our_bi_cui': '38646166C184F9C3',
 'our_bi_cui_name': 'Galileo type shuttlecraft',
 'our_bi_cui_desc': '[CLS] galileo type shuttle ##craft [unused3] the " galileo " - type shuttle ##craft was a shuttle ##craft type operated by star ##fleet in the 23rd and 24th centuries . physical arrangement . this type of shuttle ##craft had two na ##cel ##le s , attached to the sides near the ventral hull with two p ##yl ##on s each , and two impulse thrust ##er s facing aft . these thrust ##ers were above and to the side of the rear hatch . the shuttle ##s also had a small , stu ##b wings on the craft \' s center line . there was also four maneuver ##ing thrust ##er s , located near the corners of the craft . there were two side doors , [SEP]',
 'our_cross_cui': '00F7A1017770BA5D',
 'our_cross_cui_name': 'Nenebek',
 'our_cross_cui_desc': '[CLS] ne ##ne ##bek [unused3] the " ne ##ne ##bek " ( na ##r - 211 ##66 ) was a " ne ##ne ##bek " - type sub ##light mining shuttle that was in service in the independent pen ##tar ##us system in the mid - 24th century . constructed sometime in the mid - 23rd century , this shuttle was used for ferry ##ing people . " ne ##ne ##bek " was the property of captain dir ##go who had logged almost 10 , 000 hour s in this shuttle , and had modified the maneuver ##ing thrust ##er s into an unusual configuration . in 236 ##7 , the authorities on pen ##tar ##us v requested a ne ##go ##tia ##tor from the federation to [SEP]'},
 {'mention_id': '2AA8A24C22D9696B',
 'test_set_idx': 1715,
 'type': 'star_trek',
 'name': 'Klingon ship',
 'context': "[CLS] , but allowed them to get out and conduct repairs to the outer hull of ska ##la ##ar ' s shuttle without environmental suit s . ( ) ind ##ri viii was a class l planet that lacked animal life but used to be covered with deciduous plants until 236 ##9 . that year , the had to witness a [unused1] k ##ling ##on ship [unused2] destroying the planet ' s lower atmosphere including all life with a plasma reaction . ( ) the crashed on a class l planet in the ruth ##arian sector in 237 ##1 . the sole survivor was able to survive in a cave on the planet ' s surface for a while , but died when she ran out of [SEP]",
 'existing_cui': '123E018548D4EEBC',
 'existing_cui_name': 'Klingon shuttlecraft',
 'existing_cui_desc': '[CLS] k ##ling ##on shuttle ##craft [unused3] the k ##ling ##on shuttle ##craft was a type of small craft in use by the k ##ling ##on s during the mid - 22nd century . in 215 ##4 , while searching an unknown planet , the crew of discovered debris from one such craft . upon further investigation , it turned out the k ##ling ##on commander had destroyed the ship after learning the landing party it contained was infected with an unknown virus . ( ) [SEP]',
 'our_bi_cui': '416B53FB42ED940D',
 'our_bi_cui_name': 'Klingon Warbird',
 'our_bi_cui_desc': "[CLS] k ##ling ##on war ##bird [unused3] the k ##ling ##on war ##bird was a class of warship used by the k ##ling ##on empire in the mid - 22nd century , and in the mid - 23rd century of the alternate reality . in 215 ##1 , vulcan diplomat to ##s claimed that the k ##ling ##ons would have sent a squadron of war ##birds to attack earth had he not negotiated for the transport of k ##la ##ang ' s corpse to q ##o ' nos . ( ) in 225 ##8 , in the alternate reality , cadet heard a transmission that forty - seven k ##ling ##on war ##birds had been destroyed by a single rom ##ula ##n ship , which was later [SEP]",
 'our_cross_cui': 'AFE224572C159649',
 'our_cross_cui_name': "IKS Maht - H ' a",
 'our_cross_cui_desc': '[CLS] ik ##s ma ##ht - h \' a [unused3] the ik ##s " ma ##ht - h \' a " was a k ##ling ##on attack cruiser that was in service with the k ##ling ##on defense force in the mid - 24th century . the " ma ##ht - h \' a " was commanded by captain nu \' da ##q . in 236 ##9 , the " ma ##ht - h \' a " was collecting dna samples to dec ##ip ##her a four billion - year - old gene ti ##c program left by the ancient humanoid s . the ship was responsible for the era ##dication of all life on ind ##ri viii , to prevent any other parties from collecting samples [SEP]'}
]

In [10]:
for s in selected:
    for attr in ['context', 'existing_cui_desc', 'our_bi_cui_desc', 'our_cross_cui_desc']:
        split = s[attr].split()
        final = []
        for token in split:
            if token in ['[CLS]', '[SEP]']:
                continue
            if token[:2] == "##":
                final[-1] += token[2:]
            else:
                final.append(token)
        s[attr] = ' '.join(final)

In [11]:
selected

[{'mention_id': 'F5031411EF351EA5',
  'test_set_idx': 9575,
  'type': 'forgotten_realms',
  'name': 'Saukuruk',
  'context': 'they did not believe in life after death , and they believed that the world was not created — it simply always was . a very small number of iulutiuns were exceptions to this rule . among this tiny population of adherents were followers of pahluruk , ayuruk , and [unused1] saukuruk [unused2] . such iulutiuns were considered strange by the rest of their society . however , iulutiuns " did " follow a sort of world view or philosophy called " qukoko " , which held that all living creatures shared the same essence of life called " eaas " .',
  'existing_cui': 'C87DB9E9D6BA5F31',
  'existing_cui_name': 'Iulutiun',
  'existing_cui_desc': 'iulutiun [unused3] the iulutiuns ( ee - oo - loo - shee - uns ) were a human people living in the alpuk region of the great glacier . they were one of the ethnicities collectively known as the ulutiun s and the most numerous of all hu